In [1]:
from pprint import pprint
from bs4 import BeautifulSoup as bs
import requests
import re
import pandas as pd


In [2]:
def hh():
    main_link = 'https://hh.ru'
    vacancy_name = input("Введите название вакансии: ").split(' ')
    vacancy_name = "+".join(vacancy_name)

    headers = {'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/80.0.3987.149 Safari/537.36'}
    # params = {'items_on_page': '100'}
    response = requests.get(main_link + '/search/vacancy?area=&st=searchVacancy&text=' + vacancy_name, headers=headers).text
    soup = bs(response, 'lxml')

    vacancies = []

    next_button = soup.find('a', {'class': 'bloko-button HH-Pager-Controls-Next HH-Pager-Control'})
    range_len = int(next_button.find_previous_sibling().findChild('a', {'class': 'bloko-button HH-Pager-Control'}).getText())

    for i in range(0, range_len-1):
        vacancy_block = soup.find('div',{'class':'vacancy-serp'})
        vacancy_list = vacancy_block.findChildren('div',{'class':'vacancy-serp-item'}, recursive=False)

        for vacancy in vacancy_list:
            vacancy_data={}
            request = vacancy.find('span', {'class':'g-user-content'})
            if request != None:
                main_data = request.findChild()
                vacancy_name = main_data.getText()
                vacancy_data['vacancy_name'] = vacancy_name

                vacancy_link = main_data['href']
                vacancy_data['vacancy_link'] = vacancy_link

                company_name = vacancy.find('div', {'class': 'vacancy-serp-item__meta-info'}).getText()
                vacancy_data['company_name'] = company_name
                
                vacancy_city = vacancy.find('span', {'data-qa':'vacancy-serp__vacancy-address'}).getText().split(' ')
                vacancy_data['vacancy_city'] = vacancy_city[0]

                vacancy_wage = vacancy.find('div', {'class':'vacancy-serp-item__sidebar'}).getText().replace(u'\xa0', u'')

                vacancy_wage = re.split(r'\s|-', vacancy_wage)

                if vacancy_wage[0] == 'до':
                    wage_min = None
                    wage_max = int(vacancy_wage[1])
                    wage_currency = vacancy_wage[2]
                elif vacancy_wage[0] == 'от':
                    wage_min = int(vacancy_wage[1])
                    wage_max = None
                    wage_currency = vacancy_wage[2]
                elif vacancy_wage[0] == '':
                    wage_min = None
                    wage_max = None
                    wage_currency = None
                else:
                    wage_min = int(vacancy_wage[0])
                    wage_max = int(vacancy_wage[1])
                    wage_currency = vacancy_wage[2]

                vacancy_data['wage_min'] = wage_min
                vacancy_data['wage_max'] = wage_max
                vacancy_data['wage_currency'] = wage_currency

                vacancy_data['vacancy_source'] = 'hh.ru'

            vacancies.append(vacancy_data)


        next_page_block = soup.find('a', {'class': 'bloko-button HH-Pager-Controls-Next HH-Pager-Control'})
        if next_page_block is False:
            break
        else:
            next_page_link = next_page_block['href']
            response = requests.get(main_link + next_page_link, headers=headers).text
            soup = bs(response, 'lxml')

    

    df_hh = pd.DataFrame(vacancies)
    return df_hh

In [3]:
def superjob():
    main_link = 'https://russia.superjob.ru'
    vacancy_name = input("Введите название вакансии: ").split(' ')
    vacancy_name = "-".join(vacancy_name)

    headers = {'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/80.0.3987.149 Safari/537.36'}
    response = requests.get(main_link + '/vacancy/search/?keywords=' + vacancy_name, headers=headers).text
    soup = bs(response, 'lxml')

    vacancies = []

    next_button = soup.find('a', {'class': 'icMQ_ _1_Cht _3ze9n f-test-button-dalshe f-test-link-Dalshe'})
    range_len = int(next_button.previous_element)

    for i in range(range_len-1):
        vacancy_block = soup.find('div', {'style': 'display:block'})
        vacancy_list = vacancy_block.find_all('div', {'class': 'iJCa5 _2gFpt _1znz6 _2nteL'}, recursive=False)

        for vacancy in vacancy_list:
            vacancy_data = {}
            request = vacancy.find('div', {'class': '_3mfro CuJz5 PlM3e _2JVkc _3LJqf'})
            if request != None:
                main_data = request.findChild()
                vacancy_name = main_data.getText()
                vacancy_data['vacancy_name'] = vacancy_name

                vacancy_link = main_data['href']
                vacancy_data['vacancy_link'] = main_link + vacancy_link

                company_name = vacancy.find('a', {'target': '_self'}).getText()
                vacancy_data['company_name'] = company_name

                vacancy_city = vacancy.find('span', {
                    'class': '_3mfro f-test-text-company-item-location _9fXTd _2JVkc _2VHxz'}).getText().split(' ')
                vacancy_data['vacancy_city'] = vacancy_city[2]

                vacancy_wage = vacancy.find('span', {'class':'_3mfro _2Wp8I _31tpt f-test-text-company-item-salary PlM3e _2JVkc _2VHxz'}).getText().replace(u'\xa0', u' ')
                vacancy_wage = re.split(r'\s|-', vacancy_wage)

                if vacancy_wage[-1] != 'договорённости':
                    wage_currency = vacancy_wage[-1]
                else: wage_currency = None

                if vacancy_wage[0] == 'до':
                    wage_min = None
                    wage_max = int(vacancy_wage[1]+vacancy_wage[2])
                elif vacancy_wage[0] == 'от':
                    wage_min = int(vacancy_wage[1]+vacancy_wage[2])
                    wage_max = None
                elif vacancy_wage[0] == 'По':
                    wage_min = None
                    wage_max = None
                elif vacancy_wage[2] == '—':
                    wage_min = int(vacancy_wage[0]+vacancy_wage[1])
                    wage_max = int(vacancy_wage[3]+vacancy_wage[4])

                vacancy_data['wage_min'] = wage_min
                vacancy_data['wage_max'] = wage_max
                vacancy_data['wage_currency'] = wage_currency

                vacancy_data['vacancy_source'] = 'superjob.ru'

            vacancies.append(vacancy_data)

        next_page_link = soup.find('a', {'class': 'icMQ_ _1_Cht _3ze9n f-test-button-dalshe f-test-link-Dalshe'})['href']
        response = requests.get(main_link + next_page_link, headers=headers).text
        soup = bs(response, 'lxml')

    df_sj = pd.DataFrame(vacancies)
    return df_sj

In [4]:
# hh()

In [5]:
# superjob()

In [6]:
df = pd.concat([hh(),superjob()], ignore_index=True)

Введите название вакансии: python
Введите название вакансии: python


In [7]:
df

,vacancy_name,vacancy_link,company_name,vacancy_city,wage_min,wage_max,wage_currency,vacancy_source
0,Программист Python,https://hh.ru/vacancy/36615032?query=python,ООО Дефенс Групп,Москва,170000.0,250000.0,руб.,hh.ru
1,Python Developer,https://hh.ru/vacancy/35095020?query=python,УП Руна Системс,"Минск,",2500.0,NaN,USD,hh.ru
2,Data Scientist / Machine Learning Engineer,https://hh.ru/vacancy/35466744?query=python,МТС. Маркетинг,"Минск,",NaN,NaN,None,hh.ru
3,Реверс инженер мобильных приложений,https://hh.ru/vacancy/36615038?query=python,ООО Дефенс Групп,Москва,80000.0,250000.0,руб.,hh.ru
4,Старший программист,https://hh.ru/vacancy/36348947?query=python,Снежная Королева,Москва,NaN,180000.0,руб.,hh.ru
...,...,...,...,...,...,...,...,...
1985,"Системный программист, руководитель проекта",https://russia.superjob.ru/vakansii/sistemnyj-...,Акдесис,•,130000.0,150000.0,руб.,superjob.ru
1986,Инженер-программист микроконтроллеров,https://russia.superjob.ru/vakansii/inzhener-p...,Национальный исследовательский ядерный универс...,•,NaN,NaN,None,superjob.ru
1987,Аналитик,https://russia.superjob.ru/vakansii/analitik-3...,РЕСО-Гарантия,•,NaN,NaN,None,superjob.ru
1988,Системный администратор,https://russia.superjob.ru/vakansii/sistemnyj-...,ПРИМА,•,50000.0,NaN,руб.,superjob.ru
